Preprocessing the DATA


In [ ]:
import os
import cv2
import numpy as np

train_path = "change it with your train data path"
test_path = "change it with your test data path"
emotions = {'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happiness': 4, 'neutral': 5, 'sadness': 6, 'surprise': 7}

def load_data(data_path):
    images, labels = [], []
    for emotion, label in emotions.items():
        emotion_folder = os.path.join(data_path, emotion)
        for img_name in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
            img = cv2.resize(img, (48, 48)) 
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

X_train, y_train = load_data(train_path)
X_test, y_test = load_data(test_path)

X_train = X_train / 255.0
X_test = X_test / 255.0


Designing a CNN model to classify the Emotions

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128 * 6 * 6, 128) 
        self.fc2 = nn.Linear(128, 8) 
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 6 * 6) 
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x



DataLoader and Dataset

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import cv2
import os
from PIL import Image

import torch.nn as nn

class FER2013Dataset(Dataset):
    def __init__(self, data_path, emotions, transform=None):
        self.data_path = data_path
        self.emotions = emotions
        self.transform = transform
        self.images = []
        self.labels = []
        
        for emotion, label in self.emotions.items():
            emotion_folder = os.path.join(data_path, emotion)
            for img_name in os.listdir(emotion_folder):
                img_path = os.path.join(emotion_folder, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (48, 48))
                
                self.images.append(img)
                self.labels.append(label)
        
        self.images = np.array(self.images)
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label





Data Preprocessing

In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=1), 
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5], std=[0.5])  
])

train_dataset = FER2013Dataset(data_path='change it with your training data', emotions=emotions, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = FER2013Dataset(data_path='change it with your testing data', emotions=emotions, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Training Loop

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

model = EmotionCNN().to(device) 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train() 
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device) 
        
        optimizer.zero_grad() 
        
        outputs = model(images)  
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
        
        running_loss += loss.item() 
        
        _, predicted = torch.max(outputs.data, 1)  
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item() 
    
    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    
    model.eval() 
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  
            _, predicted = torch.max(outputs.data, 1)  
            
            total_val += labels.size(0)  
            correct_val += (predicted == labels).sum().item()  
    
    val_accuracy = 100 * correct_val / total_val
    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")


Testing the Model

In [ ]:
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.2f}%")


In [ ]:
torch.save(model.state_dict(), 'emotion_recognition_model.pth')
